In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

from tqdm import tqdm

from show_spectra import plot_spectra

In [2]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

In [3]:
DF = pd.read_csv('../data/main_table.csv', index_col=0)
DF = DF.loc[DF.w_median.notnull()]
Q_MEDIAN = np.load('../data/q_median.npy')
DATA_PATH = '../data/spectra'

is_nonvar = DF.nonvar_match_num == 1
is_var    = DF.var_match_num == 1
is_quasar = DF.quasar_match_num == 1

nonvar_subset = DF[is_nonvar & ~is_var & ~is_quasar]
var_subset    = DF[~is_nonvar & is_var & ~is_quasar]
quasar_subset = DF[~is_nonvar & ~is_var & is_quasar]

In [4]:
def line(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    a = (y2-y1)/(x2-x1)
    b = y1 - a*x1
    return lambda x: a * x + b

def cc_region(df):
    line_lower = line((0.7, 0.1), (3.7, 1.5))
    line_upper = line((0.7, 0.5), (2.0, 1.1))
    x = df['psfMag_u'] - df['psfMag_g']
    y = df['psfMag_g'] - df['psfMag_r']
    
    lux, llx = line_upper(x), line_lower(x)
    
    conditions = [
        (x < 0.7) & (y < -0.2),
        (x < 0.7) & (-0.2 <= y) & (y <= 0.5),
        (x < 0.7) & (y > 0.5),
        (x < 2.0) & (y > lux),
        (0.7 <= x) & (x <= 1.4) & (y < llx),
        (0.7 <= x) & (x <= 1.4) & (y > llx) & (y < lux),
        (1.4 < x) & (x < 2.0) & (y > llx) & (y < lux),
        (x >= 2.0) & (y > llx)        
    ]
    choices = [ 1, 2, 3, 3, 4, 5, 6, 7 ]
    
    return np.select(conditions, choices, default=8)

def ww_region(df):
    x = df['w_rrms']
    y = df['w_median']
    is_nonvar = df.nonvar_match_num == 1
    is_var    = df.var_match_num == 1
    is_quasar = df.quasar_match_num == 1
    
    conditions = [
        is_nonvar & ~is_var & ~is_quasar & (x > 0.1) & (x < 0.2) & (y < 0.3),
        is_nonvar & ~is_var & ~is_quasar & (x < 0.03) & (y > 0.7) & (y < 0.8),
        is_nonvar & ~is_var & ~is_quasar & (x > 0.15),
        ~is_nonvar & is_var & ~is_quasar & (x > 0.10),
        ~is_nonvar & ~is_var & is_quasar & (x > 0.10)
    ]
    choices = [ 4, 5, 1, 2, 3 ]
    
    return np.select(conditions, choices, default=6)

cc_region_names = [ f'R_CC_{n}' for n in [ 'I', 'II', 'III', 'IV', 'Va', 'Vb', 'Vc', 'VI' ] ]
ww_region_names = [ f'R_WW_{n}' for n in '123456' ]

# new

In [5]:
DF['wwr'] = ww_region(DF)
DF['ccr'] = cc_region(DF)

In [ ]:
for gid, wwr, ccr in tqdm(DF[DF.wwr != 6][['source_id', 'wwr', 'ccr']].to_numpy()):
    path = f'../figures/roi/{ww_region_names[wwr-1]}/{cc_region_names[ccr-1]}/'
    if not os.path.exists(path):
        os.makedirs(path)
        
    plot_spectra(gaia_id=gid, savefig=path, mt=DF, q_med=Q_MEDIAN, data_path=DATA_PATH)

 50%|███████████████████████████████████████████████████████████▌                                                           | 328/655 [06:41<15:07,  2.78s/it]

In [ ]:
uncl = DF[~(is_nonvar | is_var | is_quasar)]

In [ ]:
uncl[uncl.wwr == 6]

In [51]:
for gid, w_median, w_rrms, ccr in tqdm(uncl[['source_id', 'w_median', 'w_rrms', 'ccr']].to_numpy()):
    if 0.7 < w_median < 0.8 and w_rrms < 0.05:
        path = f'../figures/roi/ambi5/{cc_region_names[int(ccr-1)]}/'
    else:
        continue
        
    if not os.path.exists(path):
        os.makedirs(path)
        
    plot_spectra(gaia_id=gid, savefig=path, mt=DF, q_med=Q_MEDIAN, data_path=DATA_PATH)

HTTPError: Error 500:
No handler found for this request.

Cannot process request: 'https://gea.esac.esa.int/data-server/data' (req: Reqid: anonymous1707262210514, retrieval access: DIRECT, retrieval type: XP_CONTINUOUS, compression: null), for user: UwsJobOwner{id='anonymous', name='null', mail='null', authUsername='null', authGroups=[], pseudo='anonymous', session='C74BEE5C2D877A2F56E6DA50245CC517', ip='88.207.16.176', roles=0, parameters=Owner parameters: 6},  due to: No handler found for this request.

## old (slightly broken)

In [ ]:
%%capture
# capture the output or else the file becomes too big to commit
for gid in var_subset[var_subset.w_rrms > 0.2]['source_id']:
    plot_spectra(gaia_id=gid, savefig=True, mt=DF, q_med=Q_MEDIAN)

In [ ]:
%%capture
# capture the output or else the file becomes too big to commit
for gid in nonvar_subset[nonvar_subset.w_rrms > 0.2]['source_id']:
    plot_spectra(gaia_id=gid, savefig=True, mt=DF, q_med=Q_MEDIAN)

In [9]:
%%capture
for gid in quasar_subset[quasar_subset.w_rrms > 0.1]['source_id']:
    plot_spectra(gaia_id=gid, savefig=True, mt=DF, q_med=Q_MEDIAN)

In [13]:
%%capture
for gid in quasar_subset[quasar_subset.psfMag_r < 18]['source_id']:
    plot_spectra(gaia_id=gid, savefig=True, mt=DF, q_med=Q_MEDIAN)